In [3]:
import pandas as pd
import numpy as np 
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

In [5]:
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3787706.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.116,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3806634.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.115,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3825655.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.114,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [9]:
df = df.fillna(0)

In [10]:
df['gdp_per_capita'] = np.where(df['population'] != 0, df['gdp'] / df['population'], 0)

In [12]:
idf = df.interactive()

# 1. CO2 emission over time by contitent

In [25]:
# Define panel widgets
year_slider = pn.widgets.IntSlider(name='Year slide', 
                                   start=int(df['year'].min()), 
                                   end=int(df['year'].max()))
year_slider

IntSlider(end=2021, name='Year slide', start=1750, value=1750)

In [27]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
                                        name='Y axis', 
                                        options=['co2', 'co2_per_capita',], 
                                        button_type='success'
                                       )

In [29]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [32]:
co2_plot = co2_pipeline.hvplot(x='year', 
                               by='country', 
                               y=yaxis_co2, 
                               line_width=2, 
                               title='CO2 emisssion by continent')

In [33]:
co2_plot

# 2. Table - CO2 emission over time by continent

In [36]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, 
                             pagination='remote',
                             page_size = 10,
                             sizing_mode = 'stretch_width')
co2_table

# 3. CO2 vs GDP scatterplot

In [44]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~(idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [47]:
# co2_vs_gdp_scatterplot_pipeline

In [49]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, 
                                                                kind='scatter', 
                                                                legend=False,
                                                                height=500,
                                                                width=500)

In [50]:
co2_vs_gdp_scatterplot

# 4. Bar chart with CO2 sources by continent

In [52]:
yaxis_co2_sources = pn.widgets.RadioButtonGroup(name='Y axis', 
                                                options=['coal_co2','gas_co2','oil_co2'],
                                                button_type='success')

In [53]:
continents_excl_world = [ 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

In [56]:
co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider)&
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_sources].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [58]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', x='country', y=yaxis_co2_sources, title='CO2 source by continent')
co2_source_bar_plot

# 5. Create dashboard

In [60]:
template = pn.template.FastListTemplate(
    title='World CO2 emission dashbord',
    sidebar=[pn.pane.Markdown("# CO2 Emission and Climate Change"),
             pn.pane.Markdown("## Carbon dioxide emissions are the primary driver if global climate change"),
             pn.pane.Markdown("## Settings"),
             year_slider
            ],
    main=[
        pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=700), margin=(0,25)), 
               co2_table.panel(width=500)),
        pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
               pn.Column(yaxis_co2_sources, co2_source_bar_plot.panel(width=600)))
    ],
    accent_base_color='#88d8b0',
    header_background='#88d8b0'
)

In [ ]:
template.servable()